In [4]:
import requests
from IPython.display import display
import ipywidgets as widgets

class APIError(Exception):
    pass

class CatFactProcessor:
    def __init__(self, num_facts=5):
        self.num_facts = num_facts
        self.facts = []

    def get_fact(self):
        try:
            response = requests.get("https://catfact.ninja/fact")
            data = response.json()
            fact = data["fact"]
            self.facts.append(fact)
            return fact
        except requests.exceptions.RequestException as e:
            raise APIError(f"Error request for API: {e}")

    def get_fact_length(self):
        if not self.facts:
            return 0
        return len(self.facts[-1])

    def get_stats(self):
        if not self.facts:
            return {"average": 0, "min": 0, "max": 0}
        lengths = [len(fact) for fact in self.facts[-self.num_facts:]]
        return {
            "average": sum(lengths) / len(lengths) if lengths else 0,
            "min": min(lengths) if lengths else 0,
            "max": max(lengths) if lengths else 0,
        }

# Создаем интерактивный интерфейс для Jupyter
def create_cat_fact_interface():
    processor = CatFactProcessor()

    output = widgets.Output()
    fact_label = widgets.Label(value="")
    stats_label = widgets.Label(value="")

    def on_get_fact_clicked(b):
        with output:
            output.clear_output()
            try:
                fact = processor.get_fact()
                fact_label.value = f"Факт: {fact}"
                stats = processor.get_stats()
                stats_label.value = (
                    f"Статистика по последним {processor.num_facts} фактам:\n"
                    f"Средняя длина: {stats['average']:.1f}\n"
                    f"Минимальная длина: {stats['min']}\n"
                    f"Максимальная длина: {stats['max']}"
                )
            except APIError as e:
                fact_label.value = f"Ошибка: {str(e)}"
                stats_label.value = ""

    get_fact_button = widgets.Button(description="Получить факт о кошках")
    get_fact_button.on_click(on_get_fact_clicked)

    display(widgets.VBox([
        get_fact_button,
        fact_label,
        stats_label,
        output
    ]))

# Запускаем интерфейс
create_cat_fact_interface()

In [5]:
import unittest
from unittest.mock import patch, MagicMock

import requests

class TestCatFactProcessor(unittest.TestCase):
    def setUp(self):
        self.processor = CatFactProcessor(num_facts=3)

    @patch('requests.get')
    def test_get_fact_success(self, mock_get):
        """Test successful fact retrieval"""
        # Setup mock response
        mock_response = MagicMock()
        mock_response.json.return_value = {"fact": "Cats sleep 70% of their lives."}
        mock_response.raise_for_status.return_value = None
        mock_get.return_value = mock_response

        # Execute
        fact = self.processor.get_fact()

        # Verify
        self.assertEqual(fact, "Cats sleep 70% of their lives.")
        self.assertEqual(len(self.processor.facts), 1)
        mock_get.assert_called_once_with("https://catfact.ninja/fact")

    @patch('requests.get')
    def test_get_fact_api_error(self, mock_get):
        """Test API request failure"""
        mock_get.side_effect = requests.exceptions.RequestException("Connection error")

        with self.assertRaises(APIError) as context:
            self.processor.get_fact()

        self.assertIn("Error request for API", str(context.exception))
        self.assertEqual(len(self.processor.facts), 0)

    def test_get_fact_length_empty(self):
        """Test length calculation with no facts"""
        self.assertEqual(self.processor.get_fact_length(), 0)

    def test_get_fact_length_with_facts(self):
        """Test length calculation with existing facts"""
        self.processor.facts = ["Short", "Medium fact", "Very long fact about cats"]
        self.assertEqual(self.processor.get_fact_length(), 20)  # Length of last fact

    def test_get_stats_empty(self):
        """Test stats with no facts"""
        stats = self.processor.get_stats()
        self.assertEqual(stats, {"average": 0, "min": 0, "max": 0})

    def test_get_stats_with_facts(self):
        """Test stats calculation with facts"""
        self.processor.facts = [
            "Short",  # 5
            "Medium length fact",  # 16
            "Very long fact about cats"  # 20
        ]
        stats = self.processor.get_stats()
        self.assertEqual(stats["average"], (5 + 16 + 20) / 3)
        self.assertEqual(stats["min"], 5)
        self.assertEqual(stats["max"], 20)

    def test_get_stats_window_size(self):
        """Test stats respect num_facts window"""
        processor = CatFactProcessor(num_facts=2)
        processor.facts = [
            "1",  # 1
            "22",  # 2
            "333",  # 3
            "4444"  # 4
        ]
        stats = processor.get_stats()
        # Should only consider last 2 facts
        self.assertEqual(stats["average"], (3 + 4) / 2)
        self.assertEqual(stats["min"], 3)
        self.assertEqual(stats["max"], 4)

    @patch('requests.get')
    def test_json_decode_error(self, mock_get):
        """Test malformed JSON response"""
        mock_response = MagicMock()
        mock_response.json.side_effect = ValueError("Invalid JSON")
        mock_get.return_value = mock_response

        with self.assertRaises(APIError) as context:
            self.processor.get_fact()

        self.assertIn("Error request for API", str(context.exception))

    @patch('requests.get')
    def test_missing_fact_key(self, mock_get):
        """Test response missing 'fact' key"""
        mock_response = MagicMock()
        mock_response.json.return_value = {"error": "Not found"}
        mock_get.return_value = mock_response

        with self.assertRaises(APIError) as context:
            self.processor.get_fact()

        self.assertIn("Error request for API", str(context.exception))


if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

..F...FEE
ERROR: test_json_decode_error (__main__.TestCatFactProcessor.test_json_decode_error)
Test malformed JSON response
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/lib/python3.11/unittest/mock.py", line 1378, in patched
    return func(*newargs, **newkeywargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-5-886a4fe0c964>", line 87, in test_json_decode_error
    self.processor.get_fact()
  File "<ipython-input-4-337225e5351a>", line 16, in get_fact
    data = response.json()
           ^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/unittest/mock.py", line 1124, in __call__
    return self._mock_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/unittest/mock.py", line 1128, in _mock_call
    return self._execute_mock_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/unittest/mock.py", line 1183, in _e